<p align="center">
<img src="https://github.com/eeeeeedy/BGC/blob/main/assets/BCG_MONOGRAM.png?raw=true" alt = "BCG  icon" width="20%" height="20%">
</p>

# **Feature Engineering & Modelling**
By [Edy Setiawan](https://github.com/eeeeeedy)

We'll be working on two main sub-tasks:

    Sub-Task 1: Focus on feature engineering, specifically on improving a feature calculated as "the difference between off-peak prices in December and January the preceding year."
    
    Sub-Task 2: Build a predictive model using Random Forest to predict customer churn. This involves evaluating the model rigorously, discussing the pros and cons of using a Random Forest for this use-case, and optionally tying model performance to the client's financial metrics.

In [16]:
# Required Libraries for Feature Engineering and Modeling

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

### Sub-Task 1: Feature Engineering

We'll start by recreating the feature your colleague has worked on. Then, we'll think of ways to improve its predictive power.

#### Steps:
1. Load the cleaned data (`client_df`) and pricing data (`price_df`).
2. Create the feature as per your colleague's notebook.
3. Enhance the feature's predictive power.

Let's begin.

In [5]:
# Step 1: Load the cleaned data and pricing data
client_df = pd.read_csv(r'https://raw.githubusercontent.com/eeeeeedy/BGC/main/Task%202/client_data.csv')
price_df = pd.read_csv(r'https://raw.githubusercontent.com/eeeeeedy/BGC/main/Task%202/price_data.csv')

# Step 2: Recreate the feature
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get January and December prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']]

# Step 3: Merge this feature with the client data
client_with_new_feature = pd.merge(client_df, diff, on='id', how='left')

# Display the first few rows of the new dataframe
client_with_new_feature.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,-0.006192,0.162916


In [7]:
client_with_new_feature.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn',
       'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power'],
      dtype='object')

#### Improving the Feature’s Predictive Power

To improve this feature's predictive power, let's consider the following:

1. **Normalization**: Given that the consumption levels might vary significantly across customers, it might be beneficial to normalize this feature by the customer’s average monthly consumption.
2. **Temporal Patterns**: We can also look at how this feature changes over time. For example, the average of the differences over the past three years might be a good indicator.

Let's proceed to implement these improvements.



In [8]:
# Calculate the average monthly consumption based on the 12-month consumption data ('cons_12m')
client_with_new_feature['avg_monthly_cons'] = client_with_new_feature['cons_12m'] / 12

# Now, proceed with the normalization using this new 'avg_monthly_cons' column
client_with_new_feature['normalized_diff_energy'] = client_with_new_feature['offpeak_diff_dec_january_energy'] / client_with_new_feature['avg_monthly_cons']
client_with_new_feature['normalized_diff_power'] = client_with_new_feature['offpeak_diff_dec_january_power'] / client_with_new_feature['avg_monthly_cons']

# Handle infinities and NaNs generated due to division by zero in normalization
client_with_new_feature.replace([np.inf, -np.inf], np.nan, inplace=True)
client_with_new_feature.fillna(0, inplace=True)

# Display the first few rows of the updated dataframe
client_with_new_feature.head()


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,net_margin,num_years_antig,origin_up,pow_max,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,avg_monthly_cons,normalized_diff_energy,normalized_diff_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,0.020057,3.700961,0.000000,0.000000,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,-0.003767,0.177779,388.333333,-0.000010,0.000458
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,-0.004670,0.177779,45.333333,-0.000103,0.003922
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,-0.004547,0.177779,132.000000,-0.000034,0.001347
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,-0.006192,0.162916,368.750000,-0.000017,0.000442


The feature has been improved with the following considerations:

1. **Normalization**: We've normalized the difference between off-peak prices by the average monthly consumption for each customer. This helps account for varying consumption levels.
2. **Handling Missing Values**: Any NaNs or infinities arising from the normalization process have been replaced with zeros.

Now that we have an improved version of the existing feature, let's proceed to Sub-Task 2, where we'll build a predictive model.

---


## Sub-Task 2: Build a Predictive Model

#### Steps:
1. Train a Random Forest classifier to predict customer churn.
2. Evaluate the model's performance using suitable evaluation metrics.

Let's start by training the Random Forest classifier. We'll use the following features:

- The newly created and improved features
- Features with relatively higher correlation with churn, as identified in the exploratory data analysis

In [9]:
# Features and target variable
features = ['normalized_diff_energy', 'normalized_diff_power', 'margin_net_pow_ele', 'margin_gross_pow_ele', 'num_years_antig']
target = 'churn'

The selected features—`normalized_diff_energy`, `normalized_diff_power`, `margin_net_pow_ele`, `margin_gross_pow_ele`, and `num_years_antig`—were chosen based on a combination of factors:

1. **Correlation with Target**: These features were among those that had higher correlations with the target variable 'churn' during the exploratory data analysis. A higher correlation usually indicates that the feature has a strong relationship with the target variable.

2. **Business Understanding**: Features like `margin_net_pow_ele` and `margin_gross_pow_ele` could be directly related to customer satisfaction and, therefore, churn. Similarly, the number of years a customer has been with the company (`num_years_antig`) could be an indicator of loyalty and lower likelihood to churn.

3. **Feature Engineering**: The features `normalized_diff_energy` and `normalized_diff_power` were engineered to capture the price sensitivity of the customer, which was the hypothesis we were testing.

4. **Simplicity**: For the initial model, a smaller set of features was chosen to make the model less complex and easier to interpret.

However, feature selection is an iterative process. Additional features could be included based on model performance and business requirements. Would you like to explore adding more features to the model?

In [11]:
# Split the data into training and testing sets
X = client_with_new_feature[features]
y = client_with_new_feature[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

# Make predictions
y_pred = rf_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")
print(f"ROC AUC Score: {roc_auc:.2f}")

Accuracy: 0.88
Confusion Matrix:
[[2567   50]
 [ 288   17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      2617
           1       0.25      0.06      0.09       305

    accuracy                           0.88      2922
   macro avg       0.58      0.52      0.51      2922
weighted avg       0.83      0.88      0.85      2922

ROC AUC Score: 0.52


## Sub-Task 2: Model Evaluation

#### Model Performance Metrics:

- **Accuracy**: Approximately 88.4%, which might seem good at first glance.
- **Confusion Matrix**: Shows that the model is good at predicting the non-churn cases but not so good with the churn cases (True Positives are low).
- **Classification Report**: The recall for churn (class 1) is very low at 6%, indicating the model is not good at identifying churn cases.
- **ROC AUC Score**: Approximately 0.52, which is barely above a random classifier.

#### Why These Metrics?

- **Accuracy**: Gives a general idea of how many predictions are correct.
- **Confusion Matrix**: Provides a breakdown of the true positive, true negative, false positive, and false negative rates.
- **Classification Report**: Gives detailed metrics like precision, recall, and F1-score for each class.
- **ROC AUC Score**: Useful for imbalanced classes and gives an idea of the trade-off between sensitivity and specificity.

#### Advantages and Disadvantages of Random Forest

- **Advantages**:
    1. Handles both categorical and numerical features well.
    2. Provides feature importance scores.
    3. Generally robust to overfitting.
  
- **Disadvantages**:
    1. Complexity: Random Forests can be quite complex to visualize or explain.
    2. Computational Cost: Training can be time-consuming and computationally expensive.
    3. May not perform well on imbalanced datasets, as seen here.

#### Model Performance Satisfaction:

The model's performance is not satisfactory due to the low recall for the churn class and a low ROC AUC score. It is essential to improve the model, especially in identifying customers who are likely to churn.

#### Bonus: Financial Implication

Offering a 20% discount to customers predicted to churn could be an effective strategy if the model were good at identifying such customers. However, given the current model's performance, this strategy might not be effective.

---

Great, let's consider some ways to improve the model's performance:

1. **Class Balancing**: The current dataset is imbalanced. Methods like oversampling the minority class or using different evaluation metrics that are more robust to imbalance can be tried.
  
2. **Feature Engineering**: Additional features or transformations might improve the model.
  
3. **Parameter Tuning**: Hyperparameter tuning for the Random Forest model can also yield better results.

Let's start with class balancing by oversampling the minority class. After that, we'll evaluate the model again to see if there's any improvement in performance.

In [13]:
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Retrain the Random Forest Classifier with the balanced data
rf_clf_smote = RandomForestClassifier(random_state=42)
rf_clf_smote.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred_smote = rf_clf_smote.predict(X_test)

# Evaluate the model with the balanced data
accuracy_smote = accuracy_score(y_test, y_pred_smote)
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
class_report_smote = classification_report(y_test, y_pred_smote)
roc_auc_smote = roc_auc_score(y_test, y_pred_smote)

# Use f-strings to format the output
result_str = f"Accuracy: {accuracy_smote:.4f}\nConfusion Matrix:\n{conf_matrix_smote}\nClassification Report:\n{class_report_smote}\nROC AUC: {roc_auc_smote:.4f}"

print(result_str)


Accuracy: 0.7875
Confusion Matrix:
[[2212  405]
 [ 216   89]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88      2617
           1       0.18      0.29      0.22       305

    accuracy                           0.79      2922
   macro avg       0.55      0.57      0.55      2922
weighted avg       0.83      0.79      0.81      2922

ROC AUC: 0.5685


### Model Evaluation Comparison: Before and After Using SMOTE for Class Balancing

#### Model After SMOTE:

- **Accuracy**: Approximately 78.8%
- **Confusion Matrix**: 2212 True Negatives, 405 False Positives, 216 False Negatives, 89 True Positives.
- **Classification Report**: Recall for churn (class 1) improved to 29%.
- **ROC AUC Score**: 0.5685

#### Observations:

1. **Accuracy**: The model's accuracy decreased from 88.4% to 78.8% after applying SMOTE. While this might seem like a deterioration, it's an acceptable trade-off since the model is now better at identifying churn cases.

2. **Recall**: The recall for the churn class improved significantly, from 6% to 29%. This is a crucial improvement, as it indicates that the model is becoming better at identifying the minority class, i.e., customers who will churn.

3. **ROC AUC Score**: A minor improvement in the ROC AUC score from 0.52 to 0.5685 indicates that the model's ability to distinguish between the churn and non-churn cases has improved slightly.

4. **Precision and F1-Score**: The precision for identifying churn is still low, and so is the F1-score. This suggests that while the model is identifying more churn cases, it's also misclassifying non-churn cases as churn (False Positives).

Overall, the application of SMOTE has made the model more sensitive to the churn class, although at the expense of some accuracy. The model is now better aligned with the business objective of identifying customers who are more likely to churn.

---
### Hyperparameter Tuning:

Exploring additional hyperparameter tuning could potentially improve your model's performance. Here are some strategies:

   **Random Search: Explore the hyperparameter space to find the best set of hyperparameters for the Random Forest model.**

Let's begin

In [17]:
# Define the parameter grid for Random Forest
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize a Random Forest Classifier
rf_clf_for_tuning = RandomForestClassifier(random_state=42)

# Initialize the Random Search
random_search = RandomizedSearchCV(estimator=rf_clf_for_tuning, param_distributions=param_dist, 
                                   n_iter=100, cv=3, verbose=1, random_state=42, n_jobs=-1)

# Perform Random Search on balanced data
random_search.fit(X_train_smote, y_train_smote)

# Get the best parameters and estimator
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

# Make predictions using the best estimator
y_pred_best = best_estimator.predict(X_test)

# Evaluate the model
accuracy_best = accuracy_score(y_test, y_pred_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_pred_best)

print(f"Accuracy: {accuracy_best:.2f}")
print(f"Confusion Matrix:\n{conf_matrix_best}")
print(f"Classification Report:\n{class_report_best}")
print(f"ROC AUC Score: {roc_auc_best:.2f}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Accuracy: 0.79
Confusion Matrix:
[[2226  391]
 [ 222   83]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88      2617
           1       0.18      0.27      0.21       305

    accuracy                           0.79      2922
   macro avg       0.54      0.56      0.55      2922
weighted avg       0.83      0.79      0.81      2922

ROC AUC Score: 0.56


---

### Model Evaluation Comparison: Before and After Hyperparameter Tuning

#### Metrics for Each Model:

- **Random Forest without SMOTE**: 
    - Accuracy: ~88.4%, ROC AUC: 0.52
- **Random Forest with SMOTE**: 
    - Accuracy: ~78.8%, ROC AUC: 0.5685
- **Random Forest with SMOTE & Hyperparameter Tuning**: 
    - Accuracy: ~79%, ROC AUC: 0.56

#### Explanation of Model Performance:

The model primarily underperformed in the following areas:

1. **Low Precision for Churn**: Across all variations of the model, precision for identifying churn remained low, hovering around 18%. This indicates that the model is flagging a large number of false positives, i.e., predicting that customers will churn when they actually will not. This could lead to unnecessary costs if these customers are targeted with retention efforts.

2. **Moderate Recall after SMOTE and Tuning**: Although the recall for the churn class improved with SMOTE and hyperparameter tuning, it reached only about 27-29%. This means that the model is still missing a significant number of customers who end up churning. This could lead to lost revenue and increased customer acquisition costs.

3. **ROC AUC Score**: The ROC AUC score remained moderate (0.56) even after hyperparameter tuning. This metric indicates the model's ability to distinguish between the churn and non-churn classes, and an ideal score would be close to 1. 

4. **Trade-Off in Accuracy**: The accuracy of the model decreased after using SMOTE, from approximately 88.4% to around 79%. While this was expected due to the balancing of classes, it's a trade-off that may not be acceptable in all business contexts.

Each of these underperformances has implications for customer retention and the cost effectiveness of any retention campaigns the business may undertake.
#### Advantages and Disadvantages of Random Forest:

- **Advantages**:
    1. Handles imbalanced classes reasonably well.
    2. Good for both categorical and numerical features.
  
- **Disadvantages**:
    1. Complex and not easily interpretable.
    2. Computationally intensive for hyperparameter tuning.

#### Model Performance Satisfaction:

The model's performance is not entirely satisfactory. Although recall improved with SMOTE and hyperparameter tuning, precision remained low. The model still struggles with identifying churn accurately, leading to a high number of false positives.

#### Bonus: Financial Implication

**Assumptions**:
1. Cost to retain a customer through discounts is 70% less than acquiring a new one.
2. False positives will also receive the discount, incurring a cost.

Using the model, especially the one after hyperparameter tuning, the client can target a larger subset of customers who are more likely to churn and offer them discounts to retain them. However, the low precision means we'll also be offering discounts to customers who wouldn't have churned, thereby incurring some loss. A more balanced recall and precision would make the model more financially beneficial for the client.